In [20]:
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mido

In [25]:
########### PROCESS MIDI FILE #############
mid = MidiFile('tonghua.mid') # a Mozart piece


In [26]:

notes = []

time = float(0)
prev = float(0)

for msg in mid:
	### this time is in seconds, not ticks
	time += msg.time
	if not msg.is_meta:
		### only interested in piano channel
		if msg.channel == 0:
			if msg.type == 'note_on':
				# note in vector form to train on
				note = msg.bytes() 
				# only interested in the note and velocity. note message is in the form of [type, note, velocity]
				note = note[1:3]
				note.append(time-prev)
				prev = time
				notes.append(note)

In [27]:

######## SCALE DATA TO BETWEEN 0, 1 #######
t = []
for note in notes:
	note[0] = (note[0]-24)/88
	note[1] = note[1]/127
	t.append(note[2])
max_t = max(t) # scale based on the biggest time of any note
for note in notes:
	note[2] = note[2]/max_t

In [15]:

############ CREATE DATA, LABELS ##########
X = []
Y = []
n_prev = 30
# n_prev notes to predict the (n_prev+1)th note
for i in range(len(notes)-n_prev):
	x = notes[i:i+n_prev]
	y = notes[i+n_prev]
	X.append(x)
	Y.append(y)
# save a seed to do prediction later
seed = notes[0:n_prev]
###########################################

In [16]:

############### BUILD MODEL ###############
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(n_prev, 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=(n_prev, 3), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('linear'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='mse', optimizer='rmsprop')
model.fit(X, Y, batch_size=300, epochs=400, verbose=1)
###########################################

Build model...
Epoch 1/400
305/305 [==============================] - 0s - loss: 0.2368     
Epoch 2/400
305/305 [==============================] - 0s - loss: 0.0155     
Epoch 3/400
305/305 [==============================] - 0s - loss: 0.0418     
Epoch 4/400
305/305 [==============================] - 0s - loss: 0.0181     
Epoch 5/400
305/305 [==============================] - 0s - loss: 0.0094     
Epoch 6/400
305/305 [==============================] - 0s - loss: 0.0170     
Epoch 7/400
305/305 [==============================] - 0s - loss: 0.0115     
Epoch 8/400
305/305 [==============================] - 0s - loss: 0.0097     
Epoch 9/400
305/305 [==============================] - 0s - loss: 0.0086     
Epoch 10/400
305/305 [==============================] - 0s - loss: 0.0110     
Epoch 11/400
305/305 [==============================] - 0s - loss: 0.0093     
Epoch 12/400
305/305 [==============================] - 0s - loss: 0.0087     
Epoch 13/400
305/305 [========================

305/305 [==============================] - 0s - loss: 0.0044     
Epoch 105/400
305/305 [==============================] - 0s - loss: 0.0053     
Epoch 106/400
305/305 [==============================] - 0s - loss: 0.0117     
Epoch 107/400
305/305 [==============================] - 0s - loss: 0.0064     
Epoch 108/400
305/305 [==============================] - 0s - loss: 0.0054     
Epoch 109/400
305/305 [==============================] - 0s - loss: 0.0044     
Epoch 110/400
305/305 [==============================] - 0s - loss: 0.0061     
Epoch 111/400
305/305 [==============================] - 0s - loss: 0.0063     
Epoch 112/400
305/305 [==============================] - 0s - loss: 0.0083     
Epoch 113/400
305/305 [==============================] - 0s - loss: 0.0056     
Epoch 114/400
305/305 [==============================] - 0s - loss: 0.0061     
Epoch 115/400
305/305 [==============================] - 0s - loss: 0.0043     
Epoch 116/400
305/305 [==============================]

305/305 [==============================] - 0s - loss: 0.0051     
Epoch 207/400
305/305 [==============================] - 0s - loss: 0.0056     
Epoch 208/400
305/305 [==============================] - 0s - loss: 0.0048     
Epoch 209/400
305/305 [==============================] - 0s - loss: 0.0037     
Epoch 210/400
305/305 [==============================] - 0s - loss: 0.0045     
Epoch 211/400
305/305 [==============================] - 0s - loss: 0.0038     
Epoch 212/400
305/305 [==============================] - 0s - loss: 0.0042     
Epoch 213/400
305/305 [==============================] - 0s - loss: 0.0036     
Epoch 214/400
305/305 [==============================] - 0s - loss: 0.0043     
Epoch 215/400
305/305 [==============================] - 0s - loss: 0.0044     
Epoch 216/400
305/305 [==============================] - 0s - loss: 0.0036     
Epoch 217/400
305/305 [==============================] - 0s - loss: 0.0047     
Epoch 218/400
305/305 [==============================]

305/305 [==============================] - 0s - loss: 0.0038     
Epoch 309/400
305/305 [==============================] - 0s - loss: 0.0037     
Epoch 310/400
305/305 [==============================] - 0s - loss: 0.0034     
Epoch 311/400
305/305 [==============================] - 0s - loss: 0.0036     
Epoch 312/400
305/305 [==============================] - 0s - loss: 0.0035     
Epoch 313/400
305/305 [==============================] - 0s - loss: 0.0039     
Epoch 314/400
305/305 [==============================] - 0s - loss: 0.0032     
Epoch 315/400
305/305 [==============================] - 0s - loss: 0.0052     
Epoch 316/400
305/305 [==============================] - 0s - loss: 0.0034     
Epoch 317/400
305/305 [==============================] - 0s - loss: 0.0041     
Epoch 318/400
305/305 [==============================] - 0s - loss: 0.0036     
Epoch 319/400
305/305 [==============================] - 0s - loss: 0.0034     
Epoch 320/400
305/305 [==============================]

In [17]:

############ MAKE PREDICTIONS #############
prediction = []
x = seed
x = np.expand_dims(x, axis=0)

for i in range(3000):
	preds = model.predict(x)
	print (preds)
	x = np.squeeze(x)
	x = np.concatenate((x, preds))
	x = x[1:]
	x = np.expand_dims(x, axis=0)
	preds = np.squeeze(preds)
	prediction.append(preds)

for pred in prediction:
	pred[0] = int(88*pred[0] + 24)
	pred[1] = int(127*pred[1])
	pred[2] *= max_t
	# to reject values that will be out of range
	if pred[0] < 24:
		pred[0] = 24
	elif pred[0] > 102:
		pred[0] = 102
	if pred[1] < 0:
		pred[1] = 0
	elif pred[1] > 127:
		pred[1] = 127
	if pred[2] < 0:
		pred[2] = 0
###########################################


[[ 0.5134272   0.62080777  0.04906601]]
[[ 0.51513052  0.62093824  0.04873804]]
[[ 0.51768458  0.62115043  0.04827835]]
[[ 0.5203104   0.62136704  0.04779166]]
[[ 0.52268952  0.62155181  0.04733478]]
[[ 0.5247162   0.62169516  0.04693536]]
[[ 0.5263803   0.62179708  0.04660229]]
[[ 0.52771348  0.62186354  0.04633333]]
[[ 0.5287587   0.62189984  0.04612226]]
[[ 0.52956355  0.62191206  0.04596027]]
[[ 0.53017169  0.62190592  0.04583858]]
[[ 0.53062272  0.62188631  0.04574887]]
[[ 0.53095192  0.62185836  0.04568334]]
[[ 0.53118634  0.62182504  0.04563671]]
[[ 0.53134841  0.6217891   0.04560423]]
[[ 0.53145528  0.62175173  0.04558261]]
[[ 0.53152388  0.62171626  0.04556809]]
[[ 0.53156388  0.62168312  0.04555903]]
[[ 0.53158402  0.62165266  0.04555376]]
[[ 0.5315907   0.62162536  0.04555107]]
[[ 0.53158528  0.62159932  0.04555114]]
[[ 0.53157771  0.62157822  0.04555102]]
[[ 0.5315671   0.62156004  0.04555164]]
[[ 0.53155494  0.62154454  0.04555263]]
[[ 0.53154218  0.62153137  0.04555406]]


[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145802  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.6214717   0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.6214717   0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5

...

[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556289]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556289]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.04556291]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 

[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.6214717   0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579  0.6214717  0.0455629]]
[[ 0.5314579   0.6214717   0.04556291]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.5314

[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.04556291]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147182  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.6214717   0.0455629 ]]
[[ 0.53145796  0.62147176  0.0455629 ]]
[[ 0.5314579   0.62147176  0.0455629 ]]
[[ 0.53145796  0.62147176  0.04556291]]
[[ 0.5314579   0.62147176  0.0455629 ]]


In [18]:

###### SAVING TRACK FROM BYTES DATA #######
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for note in prediction:
	# 147 means note_on
	note = np.insert(note, 0, 147)
	bytes = note.astype(int)
	print (note)
	msg = Message.from_bytes(bytes[0:3]) 
	time = int(note[3]/0.001025) # to rescale to midi's delta ticks. arbitrary value for now.
	msg.time = time
	track.append(msg)

mid.save('shanghaitan_new.mid')

[ 147.           69.           78.            0.60981983]
[ 147.           69.           78.            0.60574359]
[ 147.           69.           78.            0.60003036]
[ 147.           69.           78.            0.59398144]
[ 147.           69.           78.            0.58830303]
[ 147.           70.           78.            0.58333898]
[ 147.           70.           78.            0.57919931]
[ 147.           70.           78.            0.57585651]
[ 147.           70.           78.            0.57323325]
[ 147.           70.           78.            0.57121986]
[ 147.           70.           78.            0.56970745]
[ 147.           70.           78.            0.56859261]
[ 147.           70.           78.            0.56777805]
[ 147.           70.           78.            0.56719857]
[ 147.           70.           78.            0.56679487]
[ 147.           70.           78.            0.56652611]
[ 147.           70.           78.            0.56634575]
[ 147.        

[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]


[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.    

[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.566281

[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 14

...

[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628108]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628108]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628108]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.

[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628108]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.    

[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.  

[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628108]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 14

[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628114]
[ 147.           70.           78.            0.56628126]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.          70.          78.           0.5662812]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628126]
[ 147.           70.           78.            0.56628132]
[ 147.           70.           78.            0.56628114]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5662812]
[ 147.          70.          78.           0.5